In [ ]:
# Get current position of the robot to plot
def odom_callback(msg):
    
    x_odom.append(msg.pose.pose.position.x)
    y_odom.append(msg.pose.pose.position.y)

In [ ]:
# Update fileds of the custom service
def update_var(var):
    rospy.wait_for_service('service')
    
    get_var = rospy.ServiceProxy('service', Callbacks_srv)
    response = get_var(var)
    
    return response   

In [ ]:
# Let the robot move autonomously in the environment
def auto_move():
	canc = GoalID()
	canc = {}
	
	print("You chose modality 1! \n")
	
	res = update_var(req)
	
	# Ask the user a new target position
	print("Set x and y of new target position: ")
	x_pos = widgets.FloatText(
		value = None,
		description = 'x: ',
		disabled = False
	)
	y_pos = widgets.FloatText(
		value = None,
		description = 'y: ',
		disabled = False
	)
	box = widgets.VBox(children=[x_pos, y_pos])  
	     
	def publish_target(change):
		target.goal.target_pose.pose.position.x = x_pos.value
		target.goal.target_pose.pose.position.y = y_pos.value
		pub_goal.publish(target)
	
	display(box, output)
	box.observe(publish_target, names='value')
	
	print(target.goal.target_pose.pose.position.y)
	# Drive the robot towards target
	while (res.s!=1):
		req.mod1 = 0
		res = update_var(req)
	
	# Control if the target is unreachable and eventually cancel it, otherwise ask the user 
	#  if he/she wants to cancel it
	if (res.re==1): # target is unreachable
		print("Goal cannot be reached!")
		pub_canc.publish(canc)
	else: # asking the user if target has to be cancelled
		choose_to_canc = widgets.RadioButtons(
			options = ['Yes', 'No'],
			layout = {'width': 'max-content'},
			description = 'Do you want to cancel this goal?',
			disabled = False,
			value = None
		)
        
		def cancelling(change):
			if (choose_to_canc.value=='Yes'): # the user wants to cancel it
				pub_canc.publish(canc)
				print("Goal cancelled")
			else:
				print("Ok, going towards the goal")
                
		display(choose_to_canc, output)
		choose_to_canc.observe(cancelling, names=['value'])

In [ ]:
# Drive the robot with keyboard inputs
def drive_alone():
    
	print("You chose modality 2! \n")
	
	res = update_var(req)
	
	print("Use keyboard to move the robot:")
	while (res.d!='q'):
		req.mod2 = 1
		res = update_var(req)

In [ ]:
# Drive the robot in the environment while being assisted for obstacle avoidance
def drive_assistance():
    
	print("You chose modality 3! \n")
	
	res = update_var(req)
	
	print("Use keyboard to move the robot:")
	while (res.d!='q'):
		req.mod1 = 0
		res = update_var(req)
		req.mod2 = 1
		res = update_var(req)

In [ ]:
# Decide control modality
def ui_decide():
	
	choose_mod = widgets.RadioButtons(
		options = ['robot moves autonomously to goal', 'drive the robot to goal', 'drive the robot with assistance to goal'],
		layout = {'width': 'max-content'},
		description = 'Choose:',
		disabled = False,
        value = None
	)
    
	def choosing(change):
		if (choose_mod.value=='robot moves autonomously to goal'):
			auto_move()
		elif (choose_mod.value=='drive the robot to goal'):
 			drive_alone()
		elif (choose_mod.value=='drive the robot with assistance to goal'):
			drive_assistance()
		else:
			print("This modality doesn't exist.")
        
	display(choose_mod, output)
	choose_mod.observe(choosing, names=['value'])

In [ ]:
# Plot current position of the robot
def plot_robot_data():

    ax_odom.plot(x_odom, y_odom)
    
    ax_odom.set_title("Robot position")
    ax_odom.set_xlabel("x [m]")
    ax_odom.set_ylabel("y [m]")

In [ ]:
# Plot position of the selected targets
def plot_target_data():
    
    x_target.append(target.goal.target_pose.pose.position.x)
    y_target.append(target.goal.target_pose.pose.position.y)
    
    ax_target.plot(x_target,y_target,'*')
    
    ax_target.set_title("Target position")
    ax_target.set_xlabel("x [m]")
    ax_target.set_ylabel("y [m]")
    
    fig.tight_layout()

In [ ]:
# Start the interface node to control the robot
def interface():
    rospy.init_node("user_interface")
    
    global pub_goal
    global pub_canc
#     global sub_odom
    
    pub_goal = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size=10)
    pub_canc = rospy.Publisher("/move_base/cancel", GoalID, queue_size=10)
    sub_odom = rospy.Subscriber("/odom", Odometry, odom_callback)    
    
    ui_decide()
    plot_robot_data()
    plot_target_data()

In [ ]:
import sys
sys.path.append('/root/ros_ws/src/RT1-III_assignment')
import final_assignment
import rospy
import matplotlib.pyplot as plt
import ipywidgets as widgets
from nav_msgs.msg import Odometry
from move_base_msgs.msg import MoveBaseActionGoal
from actionlib_msgs.msg import GoalID
from final_srv.srv import Callbacks_srv,Callbacks_srvRequest,Callbacks_srvResponse

# Define needed variables 
req = Callbacks_srvRequest()
res = Callbacks_srvResponse()
target = MoveBaseActionGoal()

output = widgets.Output()

# Initialize matplotlib widget to plot data
%matplotlib widget
global x_odom, y_odom, x_target, y_target
x_odom = []
y_odom = []
x_target = []
y_target = []
fig, axes = plt.subplots(nrows=1, ncols=2)
ax_odom, ax_target = axes.flatten()
plt.show()

# Start the interface
interface()